In [38]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# CRUD Python module file name and class name
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
#username, password and CRUD Python module name

username = "aacuser"
password = "pierogi"
# Connect to database via CRUD Module
shelter = AnimalShelter(username, password, 'nv-desktop-services.apporto.com', 32549, 'AAC', 'animals')


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Grazioso Salvare’s logo
image_filename = 'dog_logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    #unique identifier
    html.Center(html.B(html.H1('Emily Nagorski'))),
    html.Hr(),
    
    html.Div([
    #included Grazioso logo,centered and resized for display purposes
    html.A([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'10%', 'width':'10%'})
    ], href= 'https://www.snhu.edu/', target='_blank')], style={'textAlign': 'center'}),
    
    #radio buttons with labels for sorting based on requirements, displayed in-line above the table
    html.Div(
       dcc.RadioItems(
           id= 'filter-type',
           options = [
           {'label':'Water Rescue', 'value':'water'},
           {'label':'Mountain Rescue', 'value':'mountain'},
           {'label':'Disaster Rescue', 'value':'disaster'},
           {'label':'Reset', 'value':'reset'}
       ], inline = True) 
    ),
    html.Hr(),
    
    #data table features to allow single row selection, sorting by collumn, limiting page size, pagination etc
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable = "single",
                         editable = False,
                         column_selectable = False,
                         selected_rows = [0],
                         selected_columns = [],
                         sort_action = "native",
                         filter_action = "native",
                         sort_mode = "multi",
                         page_action = "native",
                         page_current = 0,
                         page_size = 10,
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
              Output('datatable-id', 'columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    #if block to filter the data table by specifically requested MongoDB queries, these specifications are are calling the values created for the radio buttons
    
    #updates request for water rescue type
    if "water" in filter_type: 
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                     {'$or': [
                                                         {'breed': 'Labrador Retriever Mix'},
                                                         {'breed': 'Chesa Bay Retr Mix'},
                                                         {'breed': 'Newfoundland Mix'},
                                                         {'breed': 'Newfoundland/Labrador Retriever'}]
                                                     },
                                                     {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                              {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                     }]
                                            })))
    #updates request for mountain rescue type    
    elif "mountain" in filter_type:
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                     {'$or': [
                                                         {'breed': 'German Shepherd'},
                                                         {'breed': 'Alaskan Malamute'},
                                                         {'breed': 'Old English Sheepdog'},
                                                         {'breed': 'Siberian Husky'},
                                                         {'breed': 'Rottweiler'}]
                                                     },
                                                     {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                              {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                     }]
                                            }))) 
    #updates request for disaster moutain rescue type                                           
    elif "disaster" in filter_type:
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                     {'$or': [
                                                         {'breed': 'Doberman Pinscher'},
                                                         {'breed': 'German Shepherd'},
                                                         {'breed': 'Golden Retriever'},
                                                         {'breed': 'Bloodhound'},
                                                         {'breed': 'Rottweiler'}]
                                                     },
                                                     {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                              {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                     }]
                                            })))
    #reset to initial view
    else:
        df = pd.DataFrame.from_records(shelter.read({}))

    df.drop(columns=['_id'],inplace=True)
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
      
    return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    #imports the data from currently displayed table, declares values for breed name and amount
    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #create pie chart from above declared values
    return [
        dcc.Graph(            
            figure = px.pie(data_frame= dff, names= names, values= values, title='Animal Breeds, by percent')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:11354/
